In [ ]:
%%capture
!pip install transformers
!pip install llama-index-core llama-index-readers-file llama-index-llms-ollama llama-index-embeddings-huggingface llama-index-llms-huggingface # Added llama-index-llms-huggingface

In [ ]:
import pandas as pd
from transformers import AutoTokenizer
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import torch
from llama_index.core.schema import Document

In [ ]:
# Initialize Tokenizer and Model
hf_token = os.getenv("HF_TOKEN")  # <-- Set your HF_TOKEN as an environment variable
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct", token=hf_token)

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [ ]:
# Only add the EOS token ID as stopping ID
stopping_ids = [tokenizer.eos_token_id]

llm = HuggingFaceLLM(
    model_name="meta-llama/Meta-Llama-3-8B-Instruct",
    model_kwargs={"token": hf_token, "torch_dtype": torch.bfloat16},
    generate_kwargs={"do_sample": True, "temperature": 0.6, "top_p": 0.9},
    tokenizer_name="meta-llama/Meta-Llama-3-8B-Instruct",
    tokenizer_kwargs={"token": hf_token},
    stopping_ids=stopping_ids,
)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [ ]:
# Load FAQ Dataset from CSV
def load_faq_from_csv(file_path):
    df = pd.read_csv(file_path)
    documents = []
    for _, row in df.iterrows():
        documents.append(Document(text=f"Q: {row['Question']} A: {row['Answer']}"))
    return documents

In [ ]:
# Assuming your CSV file is named 'asset_management_(copy).csv'
faq_documents = load_faq_from_csv("/content/BankFAQs.csv")

In [ ]:
# Set Embedding Model and LLM
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
Settings.embed_model = embed_model
Settings.llm = llm

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Create Index from Documents
index = VectorStoreIndex.from_documents(faq_documents)

In [ ]:
# Create Query Engine with Top 5 Similarity Results
query_engine = index.as_query_engine(similarity_top_k=5)

In [ ]:
# Function to Query and Retrieve Top 5 Questions and Answers
def get_top_faq_responses(query):
    response = query_engine.query(query)
    source_nodes = response.source_nodes[:5]  # Get top 5 results
    formatted_responses = []
    for node_with_score in source_nodes:
        text = node_with_score.node.text
        question = text.split("Q: ")[1].split(" A: ")[0]
        answer = text.split(" A: ")[1]
        formatted_responses.append({
            "question": question,
            "answer": answer
        })
    return formatted_responses

In [ ]:
# Example Query
query = "what is tax evasion?"
responses = get_top_faq_responses(query)
for i, response in enumerate(responses, 1):
    print(f"Q{i}: {response['question']}\nA{i}: {response['answer']}\n")


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
